In [11]:
import numpy as np
seed = 123
np.random.seed(seed)
import collections
from importlib import reload
import pandas as pd
import sklearn

In [26]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import sklearn.metrics

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import sklearn.ensemble

In [13]:
import util.data
import util.model
import util.ndcg

In [14]:
# TODO nrows
data_all = pd.read_csv('data/training_set_VU_DM_clean.csv', sep=';', nrows=10*1000)
# data_all = pd.read_csv('data/training_set_VU_DM_clean.csv', sep=';')

In [15]:
data_test_unlabelled = pd.read_csv('data/test_set_VU_DM_clean.csv', sep=';', nrows=1000)

for k in data_all.columns:
    if data_all[k].isna().sum() > 0:
#         print('rm %0.4f' % (data_all[k].isna().sum() / data_all.shape[0]), k)
        data_all.drop(columns=[k], inplace=True)

In [16]:
# # set aside some labelled data for testing (based on srch_id)
# ids = data_all.srch_id.unique()
# ids_train, ids_test = train_test_split(ids, test_size=0.5, random_state=123)
# data = data_all[data_all.srch_id.isin(ids_train)]
# data_test = data_all[data_all.srch_id.isin(ids_test)]

In [17]:
# split cross validation folds
folds = util.data.cv_folds_for_sklearn(data_all, n_cv_folds=5, resampling_ratio=0)

# Use the full (pre-splitted) dataset because we use custom indices
x, y = util.data.split_xy(data_all)

In [25]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html#sphx-glr-auto-examples-model-selection-plot-grid-search-digits-py
kwargs = {'random_state': 123}
models = {}
params = {}
models['GBoost'] = xgb.XGBRegressor(criterion='friedman_mse', loss='ls', tol=1e-4, **kwargs)
params['GBoost'] = {'objective': ['reg:linear', 'rank:ndcg'],
                    'n_estimators': [10,100,1000], 
                    'learning_rate': [0.1, 0.01, 0.001],
                    'max_depth': [1,2,3]
                   }

models['AdaBoost'] = sklearn.ensemble.AdaBoostRegressor()
params['AdaBoost'] = {'learning_rate':[1, 0.1],
                     'n_estimators': [50, 100],
                      'loss': ['linear', 'square']
                     }


scores = ['neg_mean_squared_error'] # TODO add ndcg here
for k, model in models.items():
    for score in scores:
        print("\n# %s (score: %s)" % (k, score))
        assert k in params.keys(), 'models and params should have the same keys'
        clf = sklearn.model_selection.GridSearchCV(model, params[k], cv=folds, scoring=score)
        #     util.model.cross_validation(m, x, y, folds[:n_folds_used], k, results, scoring='neg_mean_squared_error')        
        clf.fit(x, y)
        print("Best params (train)")
        print('\t', clf.best_params_)
#         y_true, y_pred = y_test, clf.predict(x_test)
#         mse = mean_squared_error(y_true, y_pred)
#         print(colored('mse: %0.4f' % mse, 'green'))

print('done')


# GBoost (score: neg_mean_squared_error)


/Users/mark/miniconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best params (train)
	 {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 100, 'objective': 'reg:linear'}

# GBoost (score: rank:ndcg)


ValueError: 'rank:ndcg' is not a valid scoring value. Use sorted(sklearn.metrics.SCORERS.keys()) to get valid options.